# **전이학습**

- torchvision.models as models를 이용해서 ImageNet으로 사전학습된 모델 사용

- 코드 출처: https://yeong-jin-data-blog.tistory.com/entry/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98-%EC%8A%A4%ED%84%B0%EB%94%94-%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5-%EB%AA%A8%EB%8D%B8-%ED%94%84%EB%A6%AC%EC%A7%95
(조금 수정)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

# GPU 사용여부 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# 데이터 불러오기 및 전처리 작업
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 
 
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# ResNet18 모델 불러오기
model = torchvision.models.resnet18(pretrained=True)
 
# 출력층 확인: fully connected layer
model.fc

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
# 모델 구조 수정
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)
model.fc

Linear(in_features=512, out_features=10, bias=True)

In [ ]:
# 모델 학습
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)
# 5 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(10):
 
    running_loss = 0.0
    for data in trainloader:
        
        inputs, labels = data[0].to(device), data[1].to(device)
          
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
 
    cost = running_loss / len(trainloader)        
    print('[%d] loss: %.3f' %(epoch + 1, cost))  
 
print('Finished Training')

[1] loss: 1.255
[2] loss: 0.868
[3] loss: 0.756
[4] loss: 0.687
[5] loss: 0.650
[6] loss: 0.621
[7] loss: 0.605
[8] loss: 0.581
[9] loss: 0.566
[10] loss: 0.543
Finished Training


In [ ]:
# 예측
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))  # 30 epoch: 84%

Accuracy of the network on the 10000 test images: 82 %


# **Model Freezing**
- 특징 추출기(feature extractor)에 대해 역전파 수행 안함. 마지막 FC 레이어의 파라미터만 업데이트
- 사전 학습 모델의 변수를 그대로 유지. 
- 학습 속도와 정확도를 향상시키거나 다른 모델과 붙여서 다른 구조 만들기 가능. (이미지 인식: 피쳐 추출은 기존 모델 사용, 분류기만 다른 방식 대체 가능)

In [ ]:
model_frz = torchvision.models.resnet18(pretrained=True)
for param in model_frz.parameters():
    param.requires_grad = False

# 마지막 레이어의 차원을 10차원으로 조절
num_features = model_frz.fc.in_features
model_frz.fc = nn.Linear(num_features, 10)
model_frz = model_frz.to(device)

# 가져온 모델의 분류기 부분 확인
model_frz.fc

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Linear(in_features=512, out_features=10, bias=True)

In [ ]:
# 모델 학습
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_frz.fc.parameters(), lr=1e-4, weight_decay=1e-2)
# 5 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(10):
 
    running_loss = 0.0
    for data in trainloader:
        
        inputs, labels = data[0].to(device), data[1].to(device)
          
        optimizer.zero_grad()
        outputs = model_frz(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
 
    cost = running_loss / len(trainloader)        
    print('[%d] loss: %.3f' %(epoch + 1, cost))  
 
print('Finished Training')

[1] loss: 2.017
[2] loss: 1.816
[3] loss: 1.776
[4] loss: 1.752
[5] loss: 1.736
[6] loss: 1.732
[7] loss: 1.726
[8] loss: 1.730
[9] loss: 1.728
[10] loss: 1.728
Finished Training


In [ ]:
# 예측
correct = 0
total = 0
with torch.no_grad():
    model_frz.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model_frz(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 41 %
